<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=c3fb3aa0e86da62e27ede52238012aa8f7e01e5ce26cde62fd3afeae1ed5c633
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-03 15:13:33
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.44 C
-------------------
Today PnL: -73.49 K (-0.51%)
Current PnL: -24.49 L (-15.82%)
CY Booked + Current PnL: -10.20 L (-6.59%)
-------------------
Total profit:  1.21 L
Total loss:  -25.70 L
-------------------
Total Booked + Current PnL: 16.78 L (13.13%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (9.94%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 89.12 L (61.98%)
Deployed:  1.28 C
Current:  1.44 C
CAGR/XIRR %: 6.71%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.22,-8.94,9.84,0.02,15641.0,-15603.0,158956.0,142.19,66.0,M-SC,14.25,234.0,-1.00,1.14,38.26,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.22,-16.73,20.21,0.09,16959.0,-16865.0,83912.0,94.66,37.0,M-SC,2.93,253.0,-0.99,0.60,10.24,OX40N,NTT,DURABLES
50,MASFIN,398.61,0.61,-1.96,24.49,22.05,23525.0,-1920.0,96060.0,-15.54,60.0,H-SC,6.74,164.0,-0.08,0.69,39.63,XR,ATH,FINANCE
43,ITC,452.00,-0.02,-2.06,12.76,10.44,25012.0,-4122.0,196016.0,-41.56,40.0,X-LC,1.51,5.0,-0.16,1.41,3.39,X40,NTT,FMCG
35,ICICIGI,2252.93,-0.70,5.25,14.20,20.19,26053.0,9150.0,183470.0,-17.43,45.0,X-MC,6.72,68.0,0.35,1.32,21.16,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,5.06,-26.58,105.38,50.79,108817.0,-37391.0,103262.0,-5.68,65.0,H-SC,6.39,151.0,-0.34,0.74,20.23,XR,ATH,IT
61,ROUTE,2227.21,4.83,-46.83,217.95,69.05,154200.0,-62316.0,70750.0,-57.12,53.0,H-SC,23.79,140.0,-0.40,0.51,9.11,SR,ATH,IT
48,KPIGREEN,730.66,3.58,-10.44,64.08,46.95,71908.0,-13081.0,112216.0,-17.35,41.0,H-SC,2.37,154.0,-0.18,0.81,40.58,MH,ATH,POWER
84,WIPRO,420.00,1.92,4.58,64.73,72.27,114704.0,7756.0,177204.0,-10.16,67.0,M-LC,3.12,99.0,0.07,1.28,11.04,XR,NTT,IT
73,TCS,4389.97,1.52,-13.50,37.90,19.28,113415.0,-46707.0,299249.0,-24.60,64.0,X-LC,6.73,1.0,-0.41,2.16,10.63,X40,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,-4.69,-39.07,85.31,12.90,113565.0,-85376.0,133120.0,95.91,47.0,H-SC,12.16,161.0,-0.75,0.96,10.69,OX40N,NTT,FINANCE
24,EASEMYTRIP,26.40,-3.40,-54.47,220.00,45.70,177109.0,-96311.0,80504.0,-16.36,62.0,M-SC,23.40,205.0,-0.54,0.58,16.03,XY24,NTT,TRAVEL
22,DIXON,18931.72,-3.17,-9.87,34.90,21.58,44081.0,-13833.0,126306.0,-54.53,28.0,X-MC,5.87,56.0,-0.31,0.91,11.82,X40N,ATH,IT
67,SONACOMS,806.63,-2.78,-14.83,63.85,39.54,55008.0,-15007.0,86152.0,-33.43,51.0,M-SC,6.42,220.0,-0.27,0.62,21.48,AR,ATH,AUTO
83,WHIRLPOOL,2270.00,-2.70,-21.25,131.02,81.92,118441.0,-24399.0,90399.0,-53.88,22.0,M-SC,4.96,236.0,-0.21,0.65,7.02,XR,NTT,DURABLES


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,0.19,0.15,106.42,106.72,131446.0,180.0,123516.0,-50.44,40.0,H-SC,1.72,139.0,0.0,0.89,23.56,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.38,-6.02,121.23,107.91,168211.0,-8890.0,138754.0,-23.39,33.0,M-SC,11.14,216.0,-0.05,1.00,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.22,-8.94,9.84,0.02,15641.0,-15603.0,158956.0,142.19,66.0,M-SC,14.25,234.0,-1.00,1.14,38.26,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.22,-16.73,20.21,0.09,16959.0,-16865.0,83912.0,94.66,37.0,M-SC,2.93,253.0,-0.99,0.60,10.24,OX40N,NTT,DURABLES
47,KANSAINER,340.00,1.02,-22.24,45.92,13.47,96294.0,-59967.0,209700.0,-68.12,37.0,H-SC,2.12,158.0,-0.62,1.51,6.63,XY24,NTT,PAINTS
66,SIS,528.00,1.11,-24.45,60.51,21.26,50757.0,-27150.0,83882.0,1978.76,47.0,H-SC,3.91,166.0,-0.53,0.60,13.33,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,1.92,4.58,64.73,72.27,114704.0,7756.0,177204.0,-10.16,67.0,M-LC,3.12,99.0,0.07,1.28,11.04,XR,NTT,IT
37,IEX,219.00,0.34,1.15,46.94,48.63,94935.0,2293.0,202247.0,-32.69,66.0,H-SC,14.78,136.0,0.02,1.46,12.83,XR,NTT,MISC
38,INDIAMART,4810.62,0.19,0.15,106.42,106.72,131446.0,180.0,123516.0,-50.44,40.0,H-SC,1.72,139.0,0.00,0.89,23.56,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,1.92,4.58,64.73,72.27,114704.0,7756.0,177204.0,-10.16,67.0,M-LC,3.12,99.0,0.07,1.28,11.04,XR,NTT,IT
37,IEX,219.00,0.34,1.15,46.94,48.63,94935.0,2293.0,202247.0,-32.69,66.0,H-SC,14.78,136.0,0.02,1.46,12.83,XR,NTT,MISC
38,INDIAMART,4810.62,0.19,0.15,106.42,106.72,131446.0,180.0,123516.0,-50.44,40.0,H-SC,1.72,139.0,0.00,0.89,23.56,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,-0.82,-1.40,37.56,35.63,77612.0,-2939.0,206635.0,-15.41,42.0,H-MC,3.64,119.0,-0.04,1.49,15.04,AR,ATH,PHARMA
25,FINCABLES,1641.55,-0.38,-6.02,121.23,107.91,168211.0,-8890.0,138754.0,-23.39,33.0,M-SC,11.14,216.0,-0.05,1.00,0.00,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,0.21,-20.48,86.69,48.45,138219.0,-41065.0,159441.0,-26.10,14.0,X-MC,8.84,64.0,-0.30,1.15,0.21,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,1.43,-36.88,114.21,35.20,92757.0,-47454.0,81216.0,6.43,26.0,X-SC,14.19,92.0,-0.51,0.58,1.43,X40,NTT,FOOTWEAR
8,AWL,485.00,-0.60,-21.54,95.64,53.50,226422.0,-65007.0,236744.0,-61.97,28.0,X-MC,1.97,58.0,-0.29,1.71,5.15,XY24,NTT,FMCG
22,DIXON,18931.72,-3.17,-9.87,34.90,21.58,44081.0,-13833.0,126306.0,-54.53,28.0,X-MC,5.87,56.0,-0.31,0.91,11.82,X40N,ATH,IT
53,PAGEIND,51605.07,-1.49,-9.50,39.30,26.07,58235.0,-15560.0,148180.0,-33.39,28.0,X-MC,10.51,55.0,-0.27,1.07,0.00,X40,ATH,APPARELS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.0,-0.72,-22.73,112.44,64.15,206732.0,-54091.0,183860.0,-55.00,46.0,X-SC,1.45,82.0,-0.26,1.32,3.14,XY24,BTT,CEMENT
43,ITC,452.0,-0.02,-2.06,12.76,10.44,25012.0,-4122.0,196016.0,-41.56,40.0,X-LC,1.51,5.0,-0.16,1.41,3.39,X40,NTT,FMCG
15,CAMS,4762.0,0.33,1.24,23.26,24.79,59311.0,3135.0,254991.0,-3.47,47.0,X-SC,1.92,86.0,0.05,1.84,24.67,X40N,NTT,MISC
8,AWL,485.0,-0.60,-21.54,95.64,53.50,226422.0,-65007.0,236744.0,-61.97,28.0,X-MC,1.97,58.0,-0.29,1.71,5.15,XY24,NTT,FMCG
36,ICICIPRULI,790.0,-0.88,1.31,29.28,30.97,52426.0,2314.0,179052.0,-22.52,47.0,X-MC,2.18,75.0,0.04,1.29,14.03,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COLPAL,3726.84,-2.58,-20.16,77.25,41.51,162426.0,-53105.0,210260.0,-4.98,30.0,X-MC,7.32,61.0,-0.33,1.51,0.00,XY25,ATH,FMCG
54,PGHH,17907.65,-0.47,-5.59,41.67,33.76,79006.0,-11220.0,189600.0,-33.45,29.0,X-MC,4.52,57.0,-0.14,1.37,0.00,X40,ATH,FMCG
55,QUESS,424.00,-0.70,-29.95,103.92,42.84,47319.0,-19472.0,45534.0,-54.27,37.0,X-SC,37.42,83.0,-0.41,0.33,0.00,XY24,NTT,MISC
53,PAGEIND,51605.07,-1.49,-9.50,39.30,26.07,58235.0,-15560.0,148180.0,-33.39,28.0,X-MC,10.51,55.0,-0.27,1.07,0.00,X40,ATH,APPARELS
9,BAJAJHFL,181.50,0.21,-20.48,86.69,48.45,138219.0,-41065.0,159441.0,-26.10,14.0,X-MC,8.84,64.0,-0.30,1.15,0.21,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-0.70,-29.95,103.92,42.84,47319.0,-19472.0,45534.0,-54.27,37.0,X-SC,37.42,83.0,-0.41,0.33,0.00,XY24,NTT,MISC
59,RELAXO,1176.00,-0.23,-47.51,190.16,52.31,144894.0,-68964.0,76196.0,-44.15,35.0,X-SC,5.02,91.0,-0.48,0.55,1.99,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,1.43,-36.88,114.21,35.20,92757.0,-47454.0,81216.0,6.43,26.0,X-SC,14.19,92.0,-0.51,0.58,1.43,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-3.17,-9.87,34.90,21.58,44081.0,-13833.0,126306.0,-54.53,28.0,X-MC,5.87,56.0,-0.31,0.91,11.82,X40N,ATH,IT
51,MEDANTA,1486.00,-0.88,1.35,22.43,24.08,28859.0,1713.0,128663.0,-6.93,41.0,X-SC,5.81,89.0,0.06,0.93,21.23,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,1.52,-13.50,37.90,19.28,113415.0,-46707.0,299249.0,-24.60,64.0,X-LC,6.73,1.0,-0.41,2.16,10.63,X40,ATH,IT
41,INFY,2275.00,1.13,7.46,44.12,54.87,148350.0,23354.0,336242.0,-16.37,64.0,X-LC,2.71,2.0,0.16,2.42,16.36,X40,BTT,IT
75,TMPV,600.00,-1.35,-15.44,68.14,42.18,159268.0,-42673.0,233737.0,-24.30,32.0,X-LC,3.13,3.0,-0.27,1.68,1.25,XY24,NTT,AUTO
81,VBL,671.64,-0.78,-3.51,40.61,35.67,123755.0,-11101.0,304741.0,-14.99,58.0,X-LC,5.39,4.0,-0.09,2.20,9.85,X40N,ATH,FMCG
43,ITC,452.00,-0.02,-2.06,12.76,10.44,25012.0,-4122.0,196016.0,-41.56,40.0,X-LC,1.51,5.0,-0.16,1.41,3.39,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-0.11,-14.98,113.70,81.67,90679.0,-14057.0,79753.0,7166.67,44.0,L-SC,18.77,271.0,-0.16,0.57,54.44,XR,NTT,CERAMICS
50,MASFIN,398.61,0.61,-1.96,24.49,22.05,23525.0,-1920.0,96060.0,-15.54,60.0,H-SC,6.74,164.0,-0.08,0.69,39.63,XR,ATH,FINANCE
69,SURYODAY,216.00,-0.76,-19.57,52.97,23.03,76290.0,-35047.0,144024.0,57.89,48.0,H-SC,9.99,167.0,-0.46,1.04,42.90,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,0.22,-8.94,9.84,0.02,15641.0,-15603.0,158956.0,142.19,66.0,M-SC,14.25,234.0,-1.00,1.14,38.26,OX40N,NTT,PAINTS
65,SIEMENS,4671.50,-0.78,-10.40,40.09,25.51,66844.0,-19360.0,166735.0,2.41,65.0,H-LC,2.71,49.0,-0.29,1.20,22.87,AR,ATH,ELECTRICAL


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.22,-8.94,9.84,0.02,15641.0,-15603.0,158956.0,142.19,66.0,M-SC,14.25,234.0,-1.00,1.14,38.26,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.04,3.31,17.07,20.95,44398.0,8325.0,260093.0,2.32,79.0,X-LC,17.34,31.0,0.19,1.87,40.12,X40,ATH,PAINTS
13,BSOFT,831.70,5.06,-26.58,105.38,50.79,108817.0,-37391.0,103262.0,-5.68,65.0,H-SC,6.39,151.0,-0.34,0.74,20.23,XR,ATH,IT
56,RAJESHEXPO,518.00,0.66,-62.94,170.04,0.07,86435.0,-86345.0,50832.0,1763.64,60.0,L-SC,1.70,268.0,-1.00,0.37,26.42,OX40N,NTT,JEWELLERY
65,SIEMENS,4671.50,-0.78,-10.40,40.09,25.51,66844.0,-19360.0,166735.0,2.41,65.0,H-LC,2.71,49.0,-0.29,1.20,22.87,AR,ATH,ELECTRICAL


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.32
1,25,45.17
2,50,76.15


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    46.30
MC    29.46
LC    24.22
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.16
X40      21.78
X40N     12.14
XR        9.89
AR        9.25
XY25      9.20
OX40N     7.73
SR        1.02
MH        0.81
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.39
X-MC    22.88
X-LC    20.74
M-SC    12.15
X-SC     8.35
H-MC     4.84
L-SC     1.41
M-MC     1.39
M-LC     1.28
H-LC     1.20
L-LC     1.00
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.03,-6.49,40.93
IT,12.39,-17.12,80.44
FINANCE,9.49,-17.66,67.38
MISC,7.62,-22.85,73.18
PAINTS,6.15,-8.01,24.06
ELECTRICAL,6.13,-10.36,50.79
INSURANCE,4.46,-2.05,38.02
PHARMA,3.98,-1.08,33.43
AUTO,3.41,-21.36,73.39


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3127081.0,21
XR,1289055.0,13
AR,1287206.0,10
X40,1014123.0,14
X40N,764218.0,9
OX40N,717384.0,10
XY25,363302.0,6
SR,278203.0,2
MH,71908.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3588811.0,25
M-SC,1371267.0,15
X-MC,1306499.0,16
X-LC,884343.0,11
X-SC,760393.0,8
H-MC,406983.0,3
L-SC,265166.0,3
M-LC,114704.0,1
H-LC,66844.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1250243.0      6
           AR         883001.0      5
M-SC       XY24       780438.0      6
H-SC       XR         773687.0      7
X-MC       X40        458978.0      7
           XY24       386772.0      3
X-LC       X40        375129.0      5
H-SC       OX40N      331769.0      4
M-SC       OX40N      299180.0      5
X-SC       X40N       297467.0      3
           XY24       282910.0      3
H-SC       SR         278203.0      2
X-MC       X40N       269066.0      4
X-LC       XY24       233066.0      2
H-MC       AR         213331.0      2
X-LC       X40N       197685.0      2
H-MC       XY24       193652.0      1
X-MC       XY25       191683.0      2
X-SC       X40        180016.0      2
L-SC       XR         178731.0      2
M-SC       XR         167619.0      2
           AR         124030.0      2
M-LC       XR         114704.0      1
L-SC       OX40N       86435.0      1
X-LC       XY25        78463.0      2
H-SC       MH          71908.0      1
H-LC       AR          66844.0      1
L-MC       XR          54314.0      1
M-MC       XY25        52667.0      1
L-LC       XY25        40489.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
